# 2. Tip Analysis:
- Assess the percentage of tips by trip.
- Investigate geographical variations in tipping and the relationship between tipping amounts and trip distances.
- Study variations in tipping by time of day, week, and notable periods (e.g., holidays).
- Explore the impact of different payment types on tipping behavior.
